In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os
from mpl_toolkits.mplot3d import axes3d
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
import sklearn.naive_bayes as nb
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from tensorflow.keras.layers import LSTM,Conv1D,MaxPooling1D
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import tensorflow.keras.backend as K 
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout,Input,Dense,Activation,Flatten,SeparableConv2D,BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
import tensorflow as tf


/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/younghwan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [2]:
##### random_forest model prediction

In [3]:
target_folder = 'swing'
mode = 'RandomForest'

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


#####here to change####
######################################################################
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [10,15,20,30,40,50,100,200,300,500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [3,4,5,6,7,8,9,10],
    'criterion' :['gini', 'entropy']
}

mod = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 10,n_jobs=-1)
######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('model best parameter : ',predict_model.best_params_)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

this model is  RandomForest 


load_file_complete
make DataFrame...
make DataFrame complete ....
label_encoding processing...
model best parameter :  {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'log2', 'n_estimators': 300}
fitting  RandomForest  is complete...
RandomForest score is : 0.9767981438515081
Validation Result...
hit:  842  miss :  20


now test new data..
testing new data result :
[answer]  -->  [predict err]
back_smash  -->  back_cut err_index :  73
back_smash  -->  back_cut err_index :  74
back_smash  -->  back_cut err_index :  75
back_smash  -->  back_cut err_index :  77
back_smash  -->  back_cut err_index :  84
back_smash  -->  back_cut err_index :  85
back_smash  -->  fo_smash err_index :  86
hit:  80  miss :  7


In [4]:
##### SVM model predict

In [4]:
target_folder = 'swing'
mode = 'SVM'
#############################################################

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


#####here to change####
######################################################################


best_score = 0

for gamma in [0.001,0.01,0.1,1,10,100]:
    for C in [0.001,0.01,0.1,1,10,100]:
        for kernel in ['linear','rbf','poly']:
            tmp_model = svm.SVC(kernel=kernel,gamma=gamma,C=C)
            scores = cross_val_score(tmp_model,d2_train_dataset,y_train,cv=10,n_jobs=-1)
            score = np.mean(scores)
            
            if score>best_score:
                
                best_score = score
                best_parameter = {'kernel':kernel,'gamma':gamma,'C':C}
                print('best_parameter is change : ',best_parameter)
            else:
                print('remain :',best_parameter)
mod = svm.SVC(**best_parameter)


######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

this model is  SVM 


load_file_complete
make DataFrame...
make DataFrame complete ....
label_encoding processing...
best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.001}
best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.01}
best_parameter is change :  {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
remain : {'kernel': 'linear', 'gamma': 0.001, 'C': 0.1}
best_parameter is change :  {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}
remain : {'kernel'

In [6]:
#### Naive Bayes model fitting

In [7]:
target_folder = 'swing'
mode = 'Naive_Bayes'


#############################################################

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


#####here to change####
######################################################################


mod = GaussianNB()


######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

this model is  Naive_Bayes 


load_file_complete
make DataFrame...
make DataFrame complete ....
label_encoding processing...
fitting  Naive_Bayes  is complete...
Naive_Bayes score is : 0.888631090487239
Validation Result...
hit:  766  miss :  96


now test new data..
testing new data result :
[answer]  -->  [predict err]
back_smash  -->  fo_drive err_index :  73
back_smash  -->  fo_drive err_index :  74
back_smash  -->  fo_drive err_index :  75
back_smash  -->  fo_drive err_index :  77
back_smash  -->  fo_drive err_index :  84
back_smash  -->  fo_drive err_index :  85
back_smash  -->  fo_drive err_index :  86
hit:  80  miss :  7


In [8]:
##############XG boost#########

In [9]:
target_folder = 'swing'
mode = 'XGBoost Classifier'


#############################################################

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))


#####here to change####
######################################################################

space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

mod =xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = space['max_depth'], gamma = space['gamma'],
                    reg_alpha = space['reg_alpha'],min_child_weight=space['min_child_weight'],
                    colsample_bytree=space['colsample_bytree'])
    
######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))

prediction = predict_model.predict(d2_test_dataset)



hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

this model is  XGBoost Classifier 


load_file_complete
make DataFrame...
make DataFrame complete ....
label_encoding processing...


/home/younghwan/.local/lib/python3.7/site-packages/xgboost/sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[10:33:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fitting  XGBoost Classifier  is complete...
XGBoost Classifier score is : 0.9721577726218097
Validation Result...
hit:  838  miss :  24


now test new data..
testing new data result :
[answer]  -->  [predict err]
back_smash  -->  back_cut err_index :  73
back_smash  -->  back_cut err_index :  74
back_smash  -->  fo_smash err_index :  75
back_smash  -->  back_cut err_index :  77
back_smash  -->  fo_drive err_index :  84
back_smash  -->  back_cut err_index :  85
back_smash  -->  back_cut err_index :  86
hit:  80  miss :  7


In [10]:
######## K-Nearest-Neighbor ############

In [11]:
target_folder = 'swing'
mode = 'KNN'


#############################################################

print('this model is ',mode,'\n\n')
filename_in_dir = []

for root, dirs, files in os.walk(target_folder):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)
print('load_file_complete')
print('make DataFrame...')
total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
print('make DataFrame complete ....')

print('label_encoding processing...')
encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

nsamples, nx, ny = x_train.shape
d2_train_dataset = x_train.reshape((nsamples,nx*ny))

nsamples, nx, ny = x_test.shape
d2_test_dataset = x_test.reshape((nsamples,nx*ny))

print('parameter Searching....')
#####here to change####
######################################################################

leaf_size = list(range(1,50))
n_neighbors = list(range(1,30))
p=[1,2]

hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

knn = KNeighborsClassifier()

mod = GridSearchCV(knn, hyperparameters, cv=10, n_jobs=-1)

######################################################################


predict_model = mod.fit(d2_train_dataset,y_train)
print('fitting ',mode,' is complete...')
print(mode,'score is :',predict_model.score(d2_test_dataset,y_test))


prediction = predict_model.predict(d2_test_dataset)

print(classification_report(prediction ,y_test))

hit = 0
miss = 0
for x in range(len(y_test)):
    if prediction[x] == y_test[x]:
        hit+=1
    else:
        miss+=1

print('Validation Result...')
print('hit: ',hit,' miss : ',miss)

print('\n\nnow test new data..')

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
sample = total_data.reshape((nsamples,nx*ny))

sample_pred = predict_model.predict(sample)

lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'err_index : ',x)

print('hit: ',hit,' miss : ',miss)

this model is  KNN 


load_file_complete
make DataFrame...
make DataFrame complete ....
label_encoding processing...
parameter Searching....
fitting  KNN  is complete...
KNN score is : 0.9860788863109049
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       103
           1       0.96      0.96      0.96        96
           2       1.00      0.99      1.00       118
           3       0.95      0.96      0.95        97
           4       1.00      1.00      1.00        85
           5       0.99      1.00      1.00       149
           6       0.99      1.00      1.00       115
           7       1.00      0.97      0.98        99

    accuracy                           0.99       862
   macro avg       0.99      0.98      0.99       862
weighted avg       0.99      0.99      0.99       862

Validation Result...
hit:  850  miss :  12


now test new data..
testing new data result :
[answer]  -->  [predict err]
back_smash  -->  fo_smash

In [12]:
#################### LSTM_ sample Code ###########

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

config = tf.ConfigProto() 
config.gpu_options.per_process_gpu_memory_fraction = 0.8 
session = tf.Session(config=config)


filename_in_dir = []

for root, dirs, files in os.walk('swing'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)

encoder = LabelEncoder()
arr_label = encoder.fit_transform(total_label)
print(encoder.classes_)

total_data = np.array(total_data)
arr_label = np.array(arr_label)
x_train,x_test,y_train,y_test = train_test_split(total_data,arr_label,test_size=0.2,random_state=0)

with tf.device('/GPU:0'):
    model = Sequential() # Sequeatial Model 
    model.add(LSTM(60, input_shape=(60,3),return_sequences = True)) # (timestep, feature) 
    model.add(Dropout(0.2))
    model.add(Conv1D(256,
                     2,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(MaxPooling1D(pool_size=4))
    model.add(LSTM(128))
    model.add(Dense(8, activation='softmax'))

    # 3. 모델 학습과정 설정하기
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    hist = model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test))
    model.save('model.h5')
    model.save_weights('model_weights.h5')
print("Best: " ,(hist.history['acc']))

filename_in_dir = []

for root, dirs, files in os.walk('test_data/'):
    for  fname in files:
        full_fname = os.path.join(root, fname)
        filename_in_dir.append(full_fname)

total_data = []
total_label = []

for z in filename_in_dir:
    sp = z.split('/')
    label = sp[1]
    f = open(z,'r')
    d = f.read()
    data = d.split('\n')
    data.pop(0)
    index = data.pop(0)
    real_data = []
    for y in range(len(data)):
        if data[y]=='':
            continue
        real_data.append(data[y].split(','))
    df = pd.DataFrame(real_data)
    index_li = index.split(',')
    df.columns = index_li
    for y in index_li:
        df[y] = pd.to_numeric(df[y],downcast='float')
    tmp_li = []

    for i in range(len(df)):
        tmp=[]
        tmp.append(df['AX'][i]/1000)
        tmp.append(df['AY'][i]/1000)
        tmp.append(df['AZ'][i]/1000)
        tmp_li.append(tmp)
    
    total_data.append(tmp_li)
    total_label.append(label)
    
total_data = np.array(total_data)
nsamples, nx, ny = total_data.shape
#sample = total_data.reshape((nsamples,nx*ny))

sample_pred = model.predict(total_data)
sample_pred = np.argmax(sample_pred,axis=-1)
lab = encoder.inverse_transform(sample_pred)

hit = 0
miss = 0
answer=[]
print('testing new data result :\n[answer]  -->  [predict err]')
for x in range(len(lab)):
    if lab[x] == total_label[x]:
        hit+=1
        answer.append(lab[x])
    else:
        miss+=1
        print(total_label[x],' --> ' ,lab[x],'        err_index number : ',x)

        
print('hit: ',hit,' miss : ',miss,'percent : ',(100*hit)/(hit+miss))



1 Physical GPUs, 1 Logical GPUs
['back_cut' 'back_drive' 'back_short' 'back_smash' 'fo_cut' 'fo_drive'
 'fo_short' 'fo_smash']
Train on 4152 samples, validate on 1039 samples
Epoch 1/100
4152/4152 [==============================] - 4s 1ms/sample - loss: 0.6495 - acc: 0.7729 - val_loss: 0.2249 - val_acc: 0.9259
Epoch 2/100
4152/4152 [==============================] - 4s 884us/sample - loss: 0.1852 - acc: 0.9403 - val_loss: 0.1483 - val_acc: 0.9490
Epoch 3/100
4152/4152 [==============================] - 4s 877us/sample - loss: 0.1317 - acc: 0.9566 - val_loss: 0.1399 - val_acc: 0.9586
Epoch 4/100
4152/4152 [==============================] - 4s 887us/sample - loss: 0.1171 - acc: 0.9622 - val_loss: 0.1303 - val_acc: 0.9577
Epoch 5/100
4152/4152 [==============================] - 4s 884us/sample - loss: 0.1086 - acc: 0.9641 - val_loss: 0.1008 - val_acc: 0.9711
Epoch 6/100
4152/4152 [==============================] - 4s 889us/sample - loss: 0.0929 - acc: 0.9701 - val_loss: 0.0981 - val_acc: 

4152/4152 [==============================] - 4s 885us/sample - loss: 0.0079 - acc: 0.9966 - val_loss: 0.0878 - val_acc: 0.9769
Epoch 59/100
4152/4152 [==============================] - 4s 886us/sample - loss: 0.0101 - acc: 0.9978 - val_loss: 0.0578 - val_acc: 0.9836
Epoch 60/100
4152/4152 [==============================] - 4s 894us/sample - loss: 0.0038 - acc: 0.9990 - val_loss: 0.0664 - val_acc: 0.9817
Epoch 61/100
4152/4152 [==============================] - 4s 888us/sample - loss: 0.0021 - acc: 0.9995 - val_loss: 0.0710 - val_acc: 0.9846
Epoch 62/100
4152/4152 [==============================] - 4s 892us/sample - loss: 0.0089 - acc: 0.9971 - val_loss: 0.0959 - val_acc: 0.9779
Epoch 63/100
4152/4152 [==============================] - 4s 886us/sample - loss: 0.0241 - acc: 0.9916 - val_loss: 0.1065 - val_acc: 0.9711
Epoch 64/100
4152/4152 [==============================] - 4s 883us/sample - loss: 0.0148 - acc: 0.9952 - val_loss: 0.0721 - val_acc: 0.9846
Epoch 65/100
4152/4152 [=========